In [63]:
import pandas as pd

# -- Selenium --
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from time import sleep
from selenium.webdriver.common.by import By


# -- API --
import re
import json
import requests

# Get today
from datetime import date

# Check time
from tqdm import tqdm

In [64]:
# driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/hệ khuyến nghị/đồ án/crawl_data_foody/chromedriver")

In [65]:
# Auto Login or Pass the block from Shopee

def login_account(driver):
    link_login = r'https://shopee.vn/buyer/login?next=https%3A%2F%2Fshopee.vn%2F'
    driver.get(link_login)

    if(len(driver.find_elements(by=By.NAME, value='loginKey'))):
        _username = ''
        _password = ''

        username_field = driver.find_element(by=By.NAME, value='loginKey')
        username_field.send_keys(_username)

        password_field = driver.find_element(by=By.NAME, value='password')
        password_field.send_keys(_password)

        sleep(0.5)

        login_button = driver.find_element(by=By.XPATH, value='//button[@class="wyhvVD _1EApiB hq6WM5 L-VL8Q cepDQ1 _7w24N1"]')
        login_button.click()

        sleep(10)

def check_login_account(driver, link):
    if(len(driver.find_elements(by=By.NAME, value='loginKey'))):
        _username = 'jax.phuchan.lmht.1999@gmail.com'
        _password = 'Quan0388925887'

        username_field = driver.find_element(by=By.NAME, value='loginKey')
        username_field.send_keys(_username)

        password_field = driver.find_element(by=By.NAME, value='password')
        password_field.send_keys(_password)

        sleep(0.5)

        login_button = driver.find_element(by=By.XPATH, value='//button[@class="wyhvVD _1EApiB hq6WM5 L-VL8Q cepDQ1 _7w24N1"]')
        login_button.click()

        sleep(5)
        driver.get(link)
        sleep(5)

In [66]:
path_data = 'C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetLink_step_1/3_link_error.csv'
data_link = pd.read_csv(path_data)
data_link = data_link.link

In [67]:
data_link

0    https://shopee.vn/Kem-gi%E1%BA%A3m-m%E1%BB%A5n...
1    https://shopee.vn/B%E1%BB%99-s%E1%BA%A3n-ph%E1...
2    https://shopee.vn/B%E1%BB%99-s%E1%BA%A3n-ph%E1...
3    https://shopee.vn/B%E1%BB%99-s%E1%BA%A3n-ph%E1...
Name: link, dtype: object

In [68]:
link_error = []

In [69]:
def startCrawl(data_link, total_product_info):
    driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/hệ khuyến nghị/đồ án/crawl_data_foody/chromedriver")
    sleep(3)
    login_account(driver)

    for link in data_link:
        driver.get(link)
        sleep(5)
        check_login_account(driver, link)
        try:
            sleep(5)
            check_login_account(driver, link)
            product_info = {}

            driver.execute_script("window.scrollTo(0, 2400);")
            sleep(1)
            driver.execute_script("window.scrollTo(0, 0);")

            # Get name_product
            if(len(driver.find_elements(by=By.XPATH, value='//div[@class="YPqix5"]')) != 0):
                product_info['name_product'] = driver.find_element(by=By.XPATH, value='//div[@class="YPqix5"]').text

            # Get discount price
            if(len(driver.find_elements(by=By.XPATH, value='//div[@class="X0xUb5"]')) != 0):
                total_price = driver.find_element(by=By.XPATH, value='//div[@class="X0xUb5"]').text.replace('₫','').replace('.','')

                if('-' in total_price):
                    list_price = total_price.split(' - ')
                    price_avg = (float(list_price[0]) + float(list_price[-1]))/2

                    product_info['price_discount'] = int(price_avg)
                else:
                    product_info['price_discount'] = int(float(total_price))

            # Get price
            if(len(driver.find_elements(by=By.XPATH, value='//div[@class="Kg2R-S"]')) != 0):
                total_price = driver.find_element(by=By.XPATH, value='//div[@class="Kg2R-S"]').text.replace('₫','').replace('.','')

                if('-' in total_price):
                    list_price = total_price.split(' - ')
                    price_avg = (float(list_price[0]) + float(list_price[-1]))/2

                    product_info['price'] = int(price_avg)
                else:
                    product_info['price'] = int(float(total_price))
            else:
                product_info['price'] = product_info['price_discount'] 



            # Get rating
            if(len(driver.find_elements(by=By.XPATH, value='//div[@class="yz-vZm _2qXJwX"]')) != 0):
                product_info['rating'] = float(driver.find_element(by=By.XPATH, value='//div[@class="yz-vZm _2qXJwX"]').text)
            else:
                product_info['rating'] = 0

            # Get total purchased
            if(driver.find_elements(by=By.XPATH, value='//div[@class="yiMptB"]') != 0):
                total_purchased = driver.find_element(by=By.XPATH, value='//div[@class="yiMptB"]').text

                if('k' in total_purchased):
                    total_purchased = int(float(total_purchased.replace('k','').replace(',','.')) * 1000)

                    product_info['purchased'] = total_purchased
                else:
                    product_info['purchased'] = int(total_purchased)


            # Get link product
            product_info['link_product'] = link

            # Get description
            if(len(driver.find_elements(by=By.XPATH, value='//div[@class="_04MILK"]')) > 1):
                product_info['description'] = driver.find_elements(by=By.XPATH, value='//div[@class="_04MILK"]')[1].text

            total_product_info.append(product_info)
            print(len(total_product_info))
            sleep(1)
        except:
            link_error.append(link)

In [70]:
import threading
import numpy as np

list_threads = []
total_product_info = []
num_thread = 1
list_partitions = [partition for partition in np.array_split(data_link, num_thread)]
for partition_list in list_partitions:
    list_threads.append(threading.Thread(target=startCrawl, args=(partition_list, total_product_info)))

In [71]:
for thread in list_threads:
    thread.start()  
    time.sleep(20)  

C:\Users\Admin\AppData\Local\Temp\ipykernel_16156\235173125.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/hệ khuyến nghị/đồ án/crawl_data_foody/chromedriver")


In [72]:
for thread in list_threads:
    thread.join()  

1
2
3
4


In [73]:
len(total_product_info)

4

In [74]:
pd.DataFrame(total_product_info).to_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/kì 9/phan tich du lieu truyen thong xa hoi/doAn/Data/GetProductInfo_step_3/ProductInfo_3_error.csv', index = False)